주의해야 될점은 word embedding의 종류를 잘 선택하는 것도 중요하다. 예를 들어 모델에서 glove를 사용한다면, 여기서도 glove를 사용하는 것이 맞지 않나 생각한다. (word embedding 종류에 따라 similairty 수치가 차이가 날 것이다.)

In [4]:
################################################################################
""" PARAMETER INIT """
################################################################################

read_file_path = '../data/conll2003/en/train.txt'
filter_incomplete_sent = False
window_size = 3
filter_none_entity = True

################################################################################

import spacy
import numpy
#import sense2vec
import random

from numpy import dot
from numpy.linalg import norm
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
nlp = spacy.load('en')
#model = sense2vec.load()

########################
### Load and Prepare Dataset
from data_handling_for_heuristic import *
#read_path = 'data/conll2003/eng.train'
#read_path = 'train.txt'
raw_data, label_data = load_conll2003(read_file_path)


# ########################
# ### 완벽한 문장만 필터링
# if filter_incomplete_sent == True:
#     sent_raw = []
#     sent_label = []
#     cnt = 0
#     for i, row in enumerate(raw_data):
#         row_nlp = nlp(row)

#     #     if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
#     #         temp = []
#     #         for token in row_nlp:
#     #             temp.append(token.pos_) # 각 token의 pos 저장
#     #         if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
#     #             if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
#     #                 sent_raw.append(raw_data[i])
#     #                 sent_label.append(label_data[i])  

#         if raw_data[i][-1] == '.' or raw_data[i][-1] == '"': # 마지막에 쉼표가 있는 문장들만 선별하자.
#             if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
#                 temp = []
#                 for token in row_nlp:
#                     temp.append(token.pos_) # 각 token의 pos 저장
#                 if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
#                     if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
#                         sent_raw.append(raw_data[i])
#                         sent_label.append(label_data[i]) 
#     #                 else:
#     #                     cnt += 1

# sent_raw = raw_data[:]
# sent_label = label_data[:]

if filter_none_entity == True:
    sent_raw, sent_label = filtering_none_entity(raw_data, label_data)
else:
    sent_raw = raw_data[:]
    sent_label = label_data[:]    

>>> [filtering_none_entity]: before = 14987 and after = 11132


In [5]:
def replace_with_word2vec(word, pos):
    if type(word) != 'str':
        word = str(word)
    num_upbound = 10
    nlp_word = nlp.vocab[word]

    ### similarity가 비슷한 단어들 추출
    queries = [w for w in nlp_word.vocab if w.is_lower == nlp_word.is_lower and w.prob >= -15]
    by_similarity = sorted(queries, key=lambda w: nlp_word.similarity(w), reverse=True)
    #print([w.lower_ for w in by_similarity[:num_upbound]])
    cand_word_list = [w.lower_ for w in by_similarity[:num_upbound]] # some candidate words
    #print(cand_word_list)
    
    ### 후보군들 중에서 어떤 것을 교체어로 선정할 것인가?
    # stemming과 lemmatization을 모두 사용하자.
    source_stem = stemmer.stem((word))
    source_lemma = lemmatiser.lemmatize((word), pos)
    
    for i, token in enumerate(cand_word_list):
        #print(source_stem, stemmer.stem(str(token)), source_lemma, lemmatiser.lemmatize(str(token), pos))
        if source_stem == stemmer.stem((token)):
            cand_word_list[i] = 'to-be-deleted'
        if source_lemma == lemmatiser.lemmatize((token), pos):
            cand_word_list[i] = 'to-be-deleted'
    cand_word_list = [x for x in cand_word_list if x != 'to-be-deleted'] # 한번에 삭제
    
    print(cand_word_list[0], nlp_word.similarity(nlp.vocab[cand_word_list[0]]))
    
    #print(cand_word_list)
    # 다 삭제하고 empty면 그냥 교체하지 말자. (ex. are: ['are', 'were', 'werent', "'re", 'is'])
    if len(cand_word_list)==0:
        return word
    else:
        return cand_word_list[0] # 후보군 중에서 가장 맨 앞에 있는 단어 선택


def replace_with_sense2vec(word, pos):
    global empty_num_sense2vec
    if type(word) != 'str':
        word = str(word)
    num_upbound = 5
    try:
        freq, query_vector = model[word+'|'+pos]
    except KeyError:
        empty_num_sense2vec += 1
        return 'sense2vec is empty!!!'
    return model.most_similar(query_vector, n=num_upbound)


def replace_with_wordnet(word, pos):
    if type(word) != 'str':
        word = str(word)   
        
    dog_synsets = wn.synsets(word,pos) # wordnet은 이렇게 POS도 인자로 넣을 수 있다
    for i, syn in enumerate(dog_synsets):
        print('%d. %s' % (i, syn.name()))
        print('alternative names (lemmas): "%s"' % '", "'.join(syn.lemma_names()))
        
        #print('definition: "%s"' % syn.definition())
        #if syn.examples():
        #    print('example usage: "%s"' % '", "'.join(syn.examples()))
        
def preprocessing_for_spacy(sent_raw):
    ### 문장 string 전체 단위
    sent_raw = sent_raw.replace("'ve", 'have')
    ### 문장 내에 있는 token 단위
    # nlp output과 sent_raw의 길이가 같도록 하기 위해 전처리 실시
    # ex. nlp()를 거치면, EU-wide와 같은 단어는 EU, -, wide로 3개로 분리된다. 
    splited_sent_raw = sent_raw.split()
    for i, token in enumerate(splited_sent_raw):
        splited_token = token.split('-')
        if not splited_token == 1: # 일반 단어들이 아니라면, result. 'EU-wide', '--'
            if not (splited_token[0] == '' or splited_token[-1] =='km'): # '-'으로만 이뤄진 단어가 아니라면, result.'EU-wide'
                filtered_token = [x for x in splited_token if x != '-'] # '-'를 list에서 삭제 result.['EU', 'wide']
                merged_token = ''.join(filtered_token)
                splited_sent_raw[i] = merged_token
    
    for i, token in enumerate(splited_sent_raw):
        if not len(token) == 1: # . 한개만 있는 token은 제외
            splited_sent_raw[i] = splited_sent_raw[i].replace('.', '')
            splited_sent_raw[i] = splited_sent_raw[i].replace('$', '')
        if token == 'cannot':
            splited_sent_raw[i] = "can"
        if token == 'dont':
            splited_sent_raw[i] = "do"
        if token == "'re":
            splited_sent_raw[i] = "are"
        if token == "**":
            splited_sent_raw[i] = "*"        
        if token == "..." or token == ".." or token == "....":
            splited_sent_raw[i] = "."         
        if token == "'m":
            splited_sent_raw[i] = "am"         
        if token == "'ll":
            splited_sent_raw[i] = "will" 
        if token == "'d":
            splited_sent_raw[i] = "would"
            
        # very specific problem of this task    
        if token == "*Note":
            splited_sent_raw[i] = "Note"   
        if token == "*Name":
            splited_sent_raw[i] = "Name" 
        if token == 'km':
            splited_sent_raw[i] = "miles"             
        if token == '237km':
            splited_sent_raw[i] = "237-km"             
    return ' '.join(splited_sent_raw)
    
    
def wordnet(word, pos):
    if type(word) != 'str':
        word = str(word)   
    synsets = wn.synsets(word, pos)
    list_synsets = []
    for i, syn in enumerate(synsets):
        #print('%d. %s' % (i, syn.name()))
        li = (syn.lemma_names())
        list_synsets += (li)
        for hyper_syn in syn.hypernyms():
            list_synsets += (hyper_syn.lemma_names())
        for hypo_syn in syn.hyponyms():
            list_synsets += (hypo_syn.lemma_names())
        for holo_syn in syn.part_holonyms():
            list_synsets += (holo_syn.lemma_names())
        for mero_syn in syn.part_meronyms():
            list_synsets += (mero_syn.lemma_names())
    
    # 똑같은 stem 제거
    source_stem = stemmer.stem(word)
    source_lemma = lemmatiser.lemmatize(word, pos)
    for i, token in enumerate(list_synsets):
        #print(source_stem, stemmer.stem(str(token)), source_lemma, lemmatiser.lemmatize(str(token), pos))
        if source_stem == stemmer.stem(token):
            list_synsets[i] = 'to-be-deleted'
        if source_lemma == lemmatiser.lemmatize(token, pos):
            list_synsets[i] = 'to-be-deleted'
        if len(token.split('_')) != 1: # 'bill_of_exchange'와 같이 명사구 또는 복합명사일 경우... 삭제...
            list_synsets[i] = 'to-be-deleted'
    list_synsets = [x for x in list_synsets if x != 'to-be-deleted'] # 한번에 삭제      
    return list_synsets
    
    
def word2vec(word, pos):
    thr = 0.1
    thr = thr / 2.0
    if type(word) != 'str':
        word = str(word)
    n_candidates = 30
    nlp_word = nlp.vocab[word]
    
    ### similarity가 비슷한 단어들 추출
    queries = [w for w in nlp_word.vocab if w.is_lower == nlp_word.is_lower and w.prob >= -11]
    by_similarity = sorted(queries, key=lambda w: nlp_word.similarity(w), reverse=True)
    #print(len([w.lower_ for w in by_similarity[:]]))
    #print(([w.lower_ for w in by_similarity[:]]))
    cand_word_list = [w.lower_ for w in by_similarity[:n_candidates]] # some candidate words
    #print(cand_word_list)
    
    ### 후보군들 중에서 어떤 것을 교체어로 선정할 것인가?
    # stemming과 lemmatization을 모두 사용하자.
    source_stem = stemmer.stem(word)
    source_lemma = lemmatiser.lemmatize(word, pos)
    
    for i, token in enumerate(cand_word_list):
        #print(source_stem, stemmer.stem(str(token)), source_lemma, lemmatiser.lemmatize(str(token), pos))
        if source_stem == stemmer.stem(token):
            cand_word_list[i] = 'to-be-deleted'
        if source_lemma == lemmatiser.lemmatize(token, pos):
            cand_word_list[i] = 'to-be-deleted'
    cand_word_list = [x for x in cand_word_list if x != 'to-be-deleted'] # 한번에 삭제

    #print(cand_word_list)
    #print('\n')
    if len(cand_word_list) == 0:
        return False
    if nlp_word.similarity(nlp.vocab[cand_word_list[0]]) > thr: # 70%로 비슷하면 
        #return cand_word_list[0] # 최상위 1개 선택
        return random.choice(cand_word_list) # 랜덤으로 선택
    else:
        list_wordnet = wordnet(word, pos)
        cand_word_list_filtered_from_wornet = [v for v in cand_word_list if v in list_wordnet]
        if len(cand_word_list_filtered_from_wornet)==0:
            if len(list_wordnet)==0:
                return False
            else:
                return list_wordnet[0]
        else:
            #print(cand_word_list_filtered_from_wornet)
            #print('\n')
            return random.choice(cand_word_list_filtered_from_wornet)
            #return cand_word_list_filtered_from_wornet[0]

----------------

In [6]:
def replace_algorithm(list_sent_raw, list_rep, pos):
    bit = False
    new_list_sent_raw = list_sent_raw[:]
    for token, _, idx in list_rep:
        
        result = word2vec(token, pos)
        if result != False: # 실패하지 않은 경우에만...
            bit = True
            new_list_sent_raw[idx] = result
        
    return new_list_sent_raw, bit

In [7]:
def window_filtering(sent_label, window_size=3):
    list_sent_label = sent_label.split()
    new_list = [0] * len(list_sent_label)
    
    ### 순방향
    for i, _ in enumerate(list_sent_label):
        temp = list_sent_label[i:i+1+window_size]
        #print(temp)
        
        tkn = False
        for each in temp:
            if each != 'O':
                tkn = True
        if tkn == True:
            new_list[i] = 1
             
    ### 역방향
    for i  in range(len(list_sent_label)-1, -1, -1):
        idx = i-window_size
        if idx < 0:
            idx = 0
        temp = list_sent_label[idx:i]
        
        tkn = False
        for each in temp:
            if each != 'O':
                tkn = True
        if tkn == True:
            new_list[i] = 1      
        
    return new_list

In [ ]:
def generation_via_replace(sent_raw, sent_label, alpha):

    #############
    """ 전처리 """
    #############
    # 동사, 명사는 context 정보의 핵심이다 모든 언어가 공통적으로 가지고 있는 특징.  
    #print(sent_raw)
    list_sent_raw = sent_raw.split()
    sent_raw = preprocessing_for_spacy(sent_raw)
    sent_nlp = nlp(sent_raw)
    noun_sorted, verb_sorted, adj_sorted, adv_sorted = [], [], [], []
    
 
    ### assert 에러 났을 때, 원인 찾기 위해 print 실시
    if not len(sent_nlp) == len(sent_label.split()):
        print(len(sent_raw.split()), len(sent_nlp), len(sent_label.split()))
        print(sent_raw)
        print(sent_nlp)
        for i, token in enumerate(sent_nlp):
            print(token, sent_raw.split()[i], sent_label.split()[i])
    assert(len(sent_nlp) == len(sent_label.split()))
    
    ### filtering with window size based on entities
    filtered_list = window_filtering(sent_label)
    
    ### 조건에 맞는 token들을 불러온다 
    for i, token in enumerate(sent_nlp):

        ### 제외되는 token들..
        if not sent_label.split()[i] == 'O': # 엔티티가 아닌 token은 제외
            continue
        is_uppercase_letter = True in map(lambda l: l.isupper(), str(token))
        if i != 0 and is_uppercase_letter == True: # 첫 번째 글자가 아니고 대문자가 하나라도 있는 token은 제외 (ex. Thursday와 같은 시간고유명사. 크게 의미적이지 않음.)
            continue
        if lemmatiser.lemmatize(str(token), 'v') == 'be': # be동사 lemma를 가지는 token은 제외
            continue
        if lemmatiser.lemmatize(str(token), 'v') == 'have': # have 동사 lemma를 가지는 token은 제외, (have 동사의 의미가 광범위하게 쓰이기도 하고, have가 분사형으로 사용되기도 하기 때문이다)
            continue
           
        if filtered_list[i] == 0: # filtering with window size based on entities
            continue
            
            
    
        ### 동사, 명사, 형용사, 부사 추출
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            pair = (sent_nlp[i], len(list(token.subtree)), i) 
            noun_sorted.append(pair) # 명사
               
        elif token.pos_ == 'VERB':
            pair = (sent_nlp[i], len(list(token.subtree)), i)
            verb_sorted.append(pair) # 동사       
        
        elif token.pos_ == 'ADJ':
            pair = (sent_nlp[i], len(list(token.subtree)), i)
            adj_sorted.append(pair) # 형용사         

        elif token.pos_ == 'ADV':
            pair = (sent_nlp[i], len(list(token.subtree)), i)
            adv_sorted.append(pair) # 부사  
            
    
    ### dependency 크기로 정렬
    noun_sorted.sort(key=lambda x: x[1], reverse=True)
    verb_sorted.sort(key=lambda x: x[1], reverse=True)
    adj_sorted.sort(key=lambda x: x[1], reverse=True)
    adv_sorted.sort(key=lambda x: x[1], reverse=True)
    
#     print((noun_sorted))
#     print((verb_sorted))
    if len(noun_sorted)==0 and len(verb_sorted)==0 and len(adj_sorted)==0 and len(adv_sorted)==0:
        return False # 교체 후보 token이 아예없다면, 새로운 문장을 만들 수 없다.

    ### thr 정하기
    noun_thr = int(len(noun_sorted) * alpha)
    verb_thr = int(len(verb_sorted) * alpha)
    adj_thr = int(len(adj_sorted) * alpha)
    adv_thr = int(len(adv_sorted) * alpha)
    
    if noun_thr == 0:
        noun_thr = 1 # 최소값은 1로 유지
    if verb_thr == 0:
        verb_thr = 1 # 최소값은 1로 유지
    if adj_thr == 0:
        adj_thr = 1 # 최소값은 1로 유지
    if adv_thr == 0:
        adv_thr = 1 # 최소값은 1로 유지        
    
    ### token list to be replaced
    noun_rep = noun_sorted[:noun_thr] 
    verb_rep = verb_sorted[:verb_thr] 
    adj_rep = adj_sorted[:adj_thr]
    adv_rep = adv_sorted[:adv_thr]
    
    
    #print(noun_rep)
    #print(verb_rep)
    #print(list_sent_raw)
    
    if len(noun_sorted) != 0: # 각자 0이 될 수도 있으니...
        list_sent_raw, bit_n = replace_algorithm(list_sent_raw, noun_rep, 'n')
        #print(list_sent_raw)
        #replaced_noun = (noun_sorted[0][0], noun_sorted[0][2]) # (token, index) pair 
        # 명사
        #print('<<< ', replaced_noun[0], '  >>>')
        #target_noun = replace_with_word2vec(replaced_noun[0], 'n')
        #print(replace_with_sense2vec(replaced_noun[0], 'NOUN'))
        #print(replace_with_wordnet(replaced_noun[0], 'n'))
        #print('replaced_noun: ', replaced_noun, ', target_noun: ', target_noun)
        #list_sent_raw[replaced_noun[1]] = target_noun # 교체
    else:
        bit_n = False
        
    
    if len(verb_sorted) != 0: # 각자 0이 될 수도 있으니...
        list_sent_raw, bit_v = replace_algorithm(list_sent_raw, verb_rep, 'v')
        #print(list_sent_raw)
        #replaced_verb = (verb_sorted[0][0], verb_sorted[0][2])
        # 동사
        #print('<<< ', replaced_verb[0], '  >>>')
        #target_verb = replace_with_word2vec(replaced_verb[0], 'v')
        #print(replace_with_sense2vec(replaced_verb[0], 'VERB'))
        #print(replace_with_wordnet(replaced_verb[0], 'v'))   
        #print('replaced_verb: ', replaced_verb, ', target_verb: ', target_verb)
        #list_sent_raw[replaced_verb[1]] = target_verb # 교체
    else:
        bit_v = False
    
    
    if len(adj_sorted) != 0: # 각자 0이 될 수도 있으니...
        list_sent_raw, bit_a = replace_algorithm(list_sent_raw, adj_rep, 'a')
    else:
        bit_a = False    
    
    
    if len(adv_sorted) != 0: # 각자 0이 될 수도 있으니...
        list_sent_raw, bit_r = replace_algorithm(list_sent_raw, adv_rep, 'r')
    else:
        bit_r = False 
        
    
    if bit_n==False and bit_v==False and bit_a==False and bit_a==False:
        return False
    else:    
        #print(' '.join(list_sent_raw))
        return ' '.join(list_sent_raw)

## Main

In [ ]:
cnt_replace_word = 0
new_sent_raw_list = []
new_sent_label_list = [] 
idx = 0

for i, row in enumerate(sent_raw):

    #if random.choice([True, False, False, False]) == False:
    #    continue
    #print(sent_raw[i])
    #print(sent_label[i])
    
    #alpha_list = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    #alpha = random.choice(alpha_list)
    alpha = 0.9
    result = generation_via_replace(sent_raw[i], sent_label[i], alpha)
    
    if result == False: # 교체어가 없었는 경우
        continue
    else:
        cnt_replace_word += 1
        new_sent_raw_list.append(result)
        new_sent_label_list.append(sent_label[i]) # 라벨은 동일하다.

#         print(', \n')
#         idx += 1
#     #     print('-------------------------------------------------------------------------')
#         if idx==10:
#             break

In [ ]:
cnt_replace_word

## Store new sent

In [ ]:
path_write = 'data_logic_warehouse/type_2/replace_1_'+str(len(new_sent_raw_list))+'.txt'

with open(path_write, 'w', encoding='UTF-8') as txt:
    
    for i, _ in enumerate(new_sent_raw_list):
        splited_sent = new_sent_raw_list[i].split()
        splited_label = new_sent_label_list[i].split()
        for j, token in enumerate(splited_sent):
            txt.write(splited_sent[j]+' '+'NNP'+' '+'B-NP'+' '+splited_label[j])
            txt.write('\n')
        txt.write('\n')

In [ ]:
float('0.2')

# 참고

worde2vec, globe 사이에 얼만큼 차이가 존재하나?

In [ ]:
def similarity(self, other):
    self = nlp.vocab[self]
    other = nlp.vocab[other]
    if self.vector_norm == 0 or other.vector_norm == 0:
        return 0.0
    return numpy.dot(self.vector, other.vector) / (self.vector_norm * other.vector_norm)

print(similarity('accumulate', 'gradually'))
print(similarity('accumulate', 'slowly'))
print(similarity('accumulate', 'steadily'))
print(similarity('accumulate', 'rapidly'))

In [ ]:
print(similarity('accuse', 'practically'))
print(similarity('accuse', 'virtually'))
print(similarity('accuse', 'falsely'))
print(similarity('accuse', 'unjustly'))
print(similarity('accuse', 'wrongly'))
print(similarity('accuse', 'angrily'))
print(similarity('accuse', 'openly'))
print(similarity('accuse', 'publicly'))

In [ ]:
print(similarity('wonder', 'wide-eyed'))


In [ ]:
def similarity(self, other):
    self = nlp.vocab[self]
    other = nlp.vocab[other]
    if self.vector_norm == 0 or other.vector_norm == 0:
        return 0.0
    return numpy.dot(self.vector, other.vector) / (self.vector_norm * other.vector_norm)

print(similarity('accumulate', 'gradually'))
print(similarity('accumulate', 'slowly'))
print(similarity('accumulate', 'steadily'))
print(similarity('accumulate', 'rapidly'))

##########################################################################################

def spacy_max_similarity(w1, w2):
    word_1 = nlp.vocab[w1]
    word_2 = nlp.vocab[w2]
    return word_1.similarity(word_2)

print(spacy_max_similarity('call', 'call'))

import numpy as np
apples = nlp.vocab[u"the"]
print(apples.prob)
print(apples.vector_norm)
# prints 1.4142135381698608, or sqrt(2)
print(np.sqrt(np.dot(apples.vector, apples.vector)))
# prints 1.0

##########################################################################################

def replace_with_word2vec2(word):
    if type(word) != 'str':
        word = str(word)
    num_upbound = 200
    word = nlp.vocab[word]
    print(word.prob)
    # Most similar ones
    #queries = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    queries = [w for w in word.vocab if w.prob >= -15]
    print(len(queries))
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    #print(by_similarity)
    #print([w.lower_ for w in by_similarity[:num_upbound]])
    cand_word_list = [w.lower_ for w in by_similarity[:num_upbound]] # some candidate words
#     sim_list = [w for w in by_similarity[:num_upbound]]
    #print(sim_list)
    print(cand_word_list)
    
replace_with_word2vec2('Sheep')

##########################################################################################

class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()
        for syn in wn.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())
        if len(antonyms) == 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []
        while i < l:
            word = sent[i]
            if word == 'not' and i+1 < l:
                ant = self.replace(sent[i+1])
                if ant:
                    words.append(ant)
                    i += 2
                    continue
            words.append(word)
            i += 1
        return words
    
replacer = AntonymReplacer()
print(replacer.replace('beautiful'))
print(replacer.replace('uglify'))

#sent = ["let's", 'not', 'uglify', 'our', 'code']
#replacer.replace_negations(sent)

##########################################################################################

synsets = wn.synsets('measures', 'n')
for i, syn in enumerate(synsets):

    print('%d. %s' % (i, syn.name()))
    print('alternative names (lemmas): "%s"' % '", "'.join(syn.lemma_names()))
    print('hypErnyms: ', syn.hypernyms())
    print('hypOrnyms: ', syn.hyponyms())
    print('part_holonyms: ', syn.part_holonyms())
    print('part_meronyms: ', syn.part_meronyms())
#     print('test: ', syn.())

In [ ]:
replace_with_word2vec2('measures')

In [ ]:
replace_with_word2vec2('beautifully')

In [ ]:
nlp = spacy.load('en')

In [ ]:
test = 'About 350 adventurers from nine countries set out on Saturday to climb , raft , bike and run in a 323-mile ( 517-km ) endurance race through the Canadian wilderness .'

In [ ]:
nlp = nlp(test)

In [ ]:
for token in nlp:
    print(token)

In [ ]:
for token in nlp:
    print(token)

In [ ]:
temp = [('c', 2), ('a',1), ('c', 3)]

In [ ]:
sorted(temp, key = lambda x: x[1])

In [ ]:
temp.sort(key=lambda x: x[1], reverse=True)

In [ ]:
doc = nlp("He is been here before.")
for token in doc:
    print(token, len(list(token.subtree)), token.pos_, lemmatiser.lemmatize((str(token)), 'v'))
    #print(np.text, np.root.text, np.root.dep_, np.root.head.text)

In [ ]:
cnt = 0
s_cnt = 0
for sent in raw_data:
    cnt += 1
    if list(sent)[-1] == '.':
        s_cnt += 1

In [ ]:
cnt

In [ ]:
s_cnt

# Replace

In [ ]:
### 어떤 것을 교체할 것인가?
# 전략1: 모두 교체
# 전략2: 전치사(closed-class words) 빼고 다 교체
# 전략3: 명사, 동사만 교체
# 전략4: dependency가 높은 명사, 동사만 교체
# 엔티티의 head
# 전략5: root token만 교체

### 어떻게 교체할 것인가?
# word2vec을 사용해서
# wordnet을 사용해서
# sense2vec 사용하면 임베딩에서 태그정보도 같이 넣을 수 있음.

# NER사전을 통해 다른 엔티티로 교체 (NER에 한해서만) - https://explosion.ai/blog/sense2vec-with-spacy
## -> 그러면 엔티티에 대한 패턴을 좀 더 확장할 수 있지 않을까

